In [23]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import gurobipy_pandas as gppd

# set intercative mode
gppd.set_interactive()

In [24]:
# importing data
hubs = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_hubs.csv")
network = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_network.csv")
plants = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_plants.csv")
roads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_roads.csv")
suppliers = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_suppliers.csv")
railroads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_railroads.csv")

In [25]:
hubs.set_index('hub', inplace=True)
hubs.head()

,index,latitude,longitude,invest,capacity
hub,,,,,
17201,1,33.64844,-95.56841,3476219,300000
17218,2,33.64124,-96.60666,3476219,300000
17359,3,29.31990,-96.10283,3476219,300000
17372,4,30.75623,-98.67770,3476219,300000
17395,5,30.84636,-96.98711,3476219,300000


In [26]:
network.head()

,counties,hubs,plants,techs,demand
0,254,33,167,1,1476310602


In [27]:
plants.set_index('plant', inplace=True)
plants.head()

,index,latitude,longitude,tech,invest,capacity,yield
plant,,,,,,,
541,1,30.220353,-97.740436,1,130956797,152063705,232
542,2,29.676724,-98.635234,1,130956797,152063705,232
543,3,33.760315,-96.559867,1,130956797,152063705,232
544,4,29.531920,-98.286901,1,130956797,152063705,232
545,5,29.688568,-98.562424,1,130956797,152063705,232


In [28]:
roads.set_index(['county','hub'], inplace=True)
roads.head()

index  distance       cost
county hub                              
48001  17201      1  219.6090  40.580691
       17218      2  264.7658  47.661277
       17359      3  370.6750  64.267840
       17372      4  420.0082  72.003286
       17395      5  199.2915  37.394907

In [29]:
# cleaning dataset by removing NA values
suppliers.dropna(inplace=True)
suppliers.county = suppliers.county.astype(int)
suppliers.set_index('county',inplace=True)
suppliers.head()

,index,supply
county,,
48001,1.0,13131.971710
48003,2.0,1177.351950
48005,3.0,3854.618542
48007,4.0,308.182629
48009,5.0,19802.136510


In [30]:
railroads.set_index(['hub','plant'], inplace=True)
railroads.head()

index     distance       cost  loading  capacity
hub   plant                                                  
17201 541        1   922.023251  17.095711  3066792    338000
      542        2  1160.842967  20.923752  3066792    338000
      543        3   186.924258   5.312809  3066792    338000
      544        4  1074.699073  19.542951  3066792    338000
      545        5  1160.842967  20.923752  3066792    338000

In [31]:
# create gurobi model
m = gp.Model()

In [32]:
### Variables
yhub = gppd.add_vars(m, hubs, name='yhub', vtype=GRB.BINARY, lb=0)
yplant = gppd.add_vars(m, plants, name='yplant', vtype=GRB.BINARY, lb=0)
ytruck = gppd.add_vars(m, roads, name='ytruck', vtype=GRB.BINARY, lb=0)
ytrain = gppd.add_vars(m, railroads, name='ytrain', vtype=GRB.BINARY, lb=0)

c_to_h = gppd.add_vars(m, roads, name='c_to_h', vtype=GRB.CONTINUOUS, lb=0) # Flow from county to hub
h_to_p = gppd.add_vars(m, railroads, name='h_to_p', vtype=GRB.CONTINUOUS, lb=0) # Flow from hub to plant

#x_out = m.addVar(vtype=GRB.CONTINUOUS, name='outside_supply', lb=0)

In [33]:
### parameters
"""
demand = network['demand]
yield = plants['yield']
cplant = plants['invest']
chub = hubs['invest']
"""
M = 500000
# fuel from external source
#c_out = True

In [34]:
### Objective
m.setObjective((plants['invest'] * yplant).sum() + (hubs['invest'] * yhub).sum() + (c_to_h * roads['cost']).sum() \
               + (ytrain * railroads['loading']).sum() + (h_to_p * railroads['cost']).sum(), 
               GRB.MINIMIZE)

In [35]:
### Highest possible demand that can be met by the network
#m.setObjective((yplant * plants['yield'] * h_to_p.groupby('plant').sum()).sum(), GRB.MAXIMIZE)

In [36]:
### Constraints
# flow conservation through hub
flow_hub = gppd.add_constrs(m,
                            c_to_h.groupby('hub').sum(),
                            GRB.EQUAL,
                            h_to_p.groupby('hub').sum(),
                            name='hub_flow'
                            )

In [37]:
# hub capacity
cap_hub = gppd.add_constrs(m,
                           c_to_h.groupby('hub').sum(),
                           GRB.LESS_EQUAL,
                           hubs['capacity']*yhub,
                           name='hub_capacity'
                           )

In [38]:
# train capacity
cap_train = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           railroads['capacity']*ytrain,
                           name='train_capacity'
                           )

In [39]:
# supplier (county) capacity
cap_county = gppd.add_constrs(m,
                           (c_to_h).groupby('county').sum(),
                           GRB.LESS_EQUAL,
                           suppliers['supply'],
                           name='suppliers_supply'
                           )

In [40]:
# delivery by truck
truck_delivery = gppd.add_constrs(m,
                           c_to_h,
                           GRB.LESS_EQUAL,
                           M*ytruck,
                           name='truck_binary'
                           )

# delivery by train
train_delivery = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           M*ytrain,
                           name='train_binary'
                           )

In [41]:
# plant capacity
cap_plant = gppd.add_constrs(m,
                            h_to_p.groupby('plant').sum(),
                            GRB.LESS_EQUAL,
                            (plants['capacity']/plants['yield'])*yplant,
                            name='plant_capacity'
                            )

In [42]:
# demand
demand_constraint = gppd.add_constrs(m,
                                     (h_to_p.groupby('plant').sum()).sum(),
                                     GRB.GREATER_EQUAL,
                                     network['demand']*0.47/232,
                                     name='network_demand'
                                     )

In [43]:
# write the model
m.write("supply_chain_biofuel.lp")

In [44]:
m.params.LogToConsole = 1
m.params.MIPgap = 0.07
m.params.ScaleFlag = 1
m.params.MIPFocus = 3
m.params.BranchDir = -1
#m.params.FlowCoverCuts = 2
m.params.Cuts = 3
m.params.NumericFocus = 1

m.update()
m.optimize()

Set parameter MIPGap to value 0.07
Set parameter ScaleFlag to value 1
Set parameter MIPFocus to value 3
Set parameter BranchDir to value -1
Set parameter Cuts to value 3
Set parameter NumericFocus to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 19892 rows, 27987 columns and 80687 nonzeros
Model fingerprint: 0xce5b2f5b
Variable types: 13894 continuous, 14093 integer (14093 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [2e+00, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 3e+06]
Presolve removed 13899 rows and 8581 columns
Presolve time: 0.06s
Presolved: 5993 rows, 19406 columns, 52307 nonzeros
Variable types: 13695 continuous, 5711 integer (5711 binary)
Found heuristic solution: objective 2.520154e+09
Root relaxation presolve 

In [ ]:
# m.computeIIS()

In [45]:
hub_to_plant = [m.getVarByName(f"h_to_p[{i},{j}]").X for i in hubs.index for j in plants.index]
county_to_hub = [m.getVarByName(f"c_to_h[{i},{j}]").X for i in suppliers.index for j in hubs.index]
yplant = [m.getVarByName(f"yplant[{i}]").X for i in plants.index]
yhub = [m.getVarByName(f"yhub[{i}]").X for i in hubs.index]
ytrain = [m.getVarByName(f"ytrain[{i},{j}]").X for i in hubs.index for j in plants.index]
ytruck = [m.getVarByName(f"ytruck[{i},{j}]").X for i in suppliers.index for j in hubs.index]

In [46]:
cost_per_litre = m.getObjective().getValue()/(sum(hub_to_plant)*232)
print(cost_per_litre)

1.173386322460577


In [47]:
sum(ytruck)

8382.0

In [48]:
sum(county_to_hub)

2990801.650603257

In [49]:
sum(hub_to_plant)*232

693865982.9399999

In [50]:
suppliers['supply'].sum()*232

708383628.3169292

In [51]:
np.nonzero(hub_to_plant)

(array([ 501, 1115, 1323, 2002, 2405, 2505, 2806, 3327, 3741, 4841, 5144],
       dtype=int64),)

In [52]:
np.nonzero(county_to_hub)

(array([  30,   40,   96,  127,  151,  187,  226,  247,  271,  300,  345,
         385,  424,  444,  490,  498,  535,  576,  608,  643,  690,  748,
         787,  795,  840,  861,  902,  935,  964,  976, 1018, 1037, 1078,
        1103, 1144, 1171, 1218, 1243, 1273, 1294, 1327, 1372, 1400, 1441,
        1468, 1488, 1537, 1554, 1598, 1632, 1672, 1690, 1719, 1756, 1862,
        1867, 1888, 1936, 1961, 1994, 2024, 2053, 2107, 2134, 2173, 2197,
        2218, 2247, 2307, 2362, 2391, 2423, 2453, 2482, 2530, 2563, 2590,
        2621, 2670, 2684, 2713, 2755, 2779, 2808, 2841, 2845, 2882, 2915,
        2959, 2984, 3033, 3052, 3080, 3124, 3157, 3183, 3187, 3223, 3256,
        3283, 3316, 3363, 3406, 3435, 3487, 3528, 3559, 3579, 3604, 3649,
        3677, 3726, 3736, 3809, 3850, 3864, 3913, 3938, 3976, 4042, 4087,
        4120, 4144, 4165, 4202, 4219, 4238, 4260, 4318, 4330, 4359, 4392,
        4429, 4466, 4516, 4543, 4568, 4594, 4635, 4681, 4697, 4730, 4735,
        4782, 4801, 4848, 4873, 4912, 

In [53]:
# roads_copy = roads.copy()
# roads_copy['c_to_h'] = c_to_h.gppd.X.to_frame()
# roads_copy = roads_copy.groupby('county').sum()

In [54]:
# roads_copy['supply'] = suppliers['supply'].copy()

In [55]:
# vio_list = []
# for index, row in roads_copy.iterrows():
#     if row['c_to_h'] > row['supply']+0.0001:
#         vio_list.append(index)

In [56]:
# roads_copy.loc[vio_list]

,index,distance,cost,c_to_h,supply
county,,,,,


In [57]:
sum(hub_to_plant)*232

693865982.9399999

In [58]:
sum(yhub)

10.0

In [59]:
yhub

[-0.0,
 -0.0,
 -0.0,
 1.0,
 0.0,
 0.0,
 -0.0,
 1.0,
 -0.0,
 0.0,
 0.0,
 1.0,
 -0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 -0.0,
 1.0,
 -0.0,
 0.0,
 1.0,
 0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0]

In [60]:
sum(yplant)

5.0